In [41]:
import json
import itertools
import sys

In [62]:
from kafka import KafkaConsumer
consumer = KafkaConsumer('test')

In [43]:
import kafka

In [44]:
from kafka import TopicPartition

In [45]:
tp = kafka.TopicPartition('test', 0)

In [20]:
TopicPartition('test', 0)

TopicPartition(topic='test', partition=0)

In [ ]:
from kafka import TopicPartition

In [28]:
consumer.beginning_offsets([tp])

{TopicPartition(topic='test', partition=0): 0}

In [29]:
consumer.end_offsets([tp])

{TopicPartition(topic='test', partition=0): 214480}

In [31]:
consumer.seek(tp, 214479)

In [63]:
for msg in consumer:
    break

In [64]:
msg.value

b'{"schema":{"type":"struct","optional":false,"fields":[{"type":"int32","optional":true,"field":"timestamp"},{"type":"int32","optional":true,"field":"order"},{"type":"string","optional":true,"field":"operation"},{"type":"string","optional":true,"field":"database"},{"type":"string","optional":true,"field":"object"}],"name":"mongodbschema_mirth_YEuSnlikDh"},"payload":{"timestamp":1549306230,"order":4,"operation":"i","database":"mirth.YEuSnlikDh","object":"{\\"_id\\":{\\"$oid\\":\\"5c5889764b4aef638b3918a8\\"},\\"address\\":\\"965 Rhodes Roads\\\\nNew Elizabeth, NH 78952\\",\\"birthdate\\":\\"1982-12-02\\",\\"blood_group\\":\\"O-\\",\\"company\\":\\"Smith, Harris and Aguirre\\",\\"counter\\":10314,\\"created\\":\\"2019-02-04T13:50:30.321766\\",\\"current_location\\":[-5.5977115,-149.340818],\\"job\\":\\"Horticulturist, commercial\\",\\"mail\\":\\"jacksondavid@hotmail.com\\",\\"name\\":\\"Craig Carter\\",\\"residence\\":\\"69643 Richard Village\\\\nAnthonychester, OH 37919\\",\\"sex\\":\\"

In [65]:
import json

In [66]:
d = json.loads(msg.value)

In [67]:
d

{'schema': {'type': 'struct',
  'optional': False,
  'fields': [{'type': 'int32', 'optional': True, 'field': 'timestamp'},
   {'type': 'int32', 'optional': True, 'field': 'order'},
   {'type': 'string', 'optional': True, 'field': 'operation'},
   {'type': 'string', 'optional': True, 'field': 'database'},
   {'type': 'string', 'optional': True, 'field': 'object'}],
  'name': 'mongodbschema_mirth_YEuSnlikDh'},
 'payload': {'timestamp': 1549306230,
  'order': 4,
  'operation': 'i',
  'database': 'mirth.YEuSnlikDh',
  'object': '{"_id":{"$oid":"5c5889764b4aef638b3918a8"},"address":"965 Rhodes Roads\\nNew Elizabeth, NH 78952","birthdate":"1982-12-02","blood_group":"O-","company":"Smith, Harris and Aguirre","counter":10314,"created":"2019-02-04T13:50:30.321766","current_location":[-5.5977115,-149.340818],"job":"Horticulturist, commercial","mail":"jacksondavid@hotmail.com","name":"Craig Carter","residence":"69643 Richard Village\\nAnthonychester, OH 37919","sex":"M","ssn":"765-58-8507","usern

In [68]:
json.loads(d['payload']['object'])

{'_id': {'$oid': '5c5889764b4aef638b3918a8'},
 'address': '965 Rhodes Roads\nNew Elizabeth, NH 78952',
 'birthdate': '1982-12-02',
 'blood_group': 'O-',
 'company': 'Smith, Harris and Aguirre',
 'counter': 10314,
 'created': '2019-02-04T13:50:30.321766',
 'current_location': [-5.5977115, -149.340818],
 'job': 'Horticulturist, commercial',
 'mail': 'jacksondavid@hotmail.com',
 'name': 'Craig Carter',
 'residence': '69643 Richard Village\nAnthonychester, OH 37919',
 'sex': 'M',
 'ssn': '765-58-8507',
 'username': 'mbenson',
 'website': ['http://byrd.com/',
  'https://www.scott.com/',
  'https://www.clark-whitaker.com/']}

In [40]:
d['ns']

{'coll': 'APYqkeyekg', 'db': 'mirth'}

In [52]:

data = []
counts = set()
first = None

for i, msg in enumerate(consumer, 1):
    recieved_time = datetime.datetime.now()
    if i % 100 == 0:
        sys.stdout.write('|')
    elif i % 10 == 0:
        sys.stdout.write('.')
    if i % 1000 == 0:
        sys.stdout.write(f' {i}\n')
    doc = json.loads(msg.value)['fullDocument']
    if first is None:
        first = doc['counter']
    if first <= doc['counter'] < first + 10000:
        kafka_time = datetime.datetime.fromtimestamp(msg.timestamp / 1000)
        created_time = datetime.datetime.strptime(doc['created'], '%Y-%m-%dT%H:%M:%S.%f')
        data.append((created_time, kafka_time, recieved_time, doc['counter']))
        counts.add(doc['counter'])
        if len(counts) == 10000:
            break

.........|.........|.........|.........|.........|.........|.........|.........|.........|.........| 1000
.........|.........|.........|.........|.........|.........|.........|.........|.........|.........| 2000
.........|.........|.........|.........|.........|.........|.........|.........|.........|.........| 3000
.........|.........|.........|.........|.........|.........|.........|.........|.........|.........| 4000
.........|.........|.........|.........|.........|.........|.........|.........|.........|.........| 5000
.........|.........|.........|.........|.........|.........|.........|.........|.........|.........| 6000
.........|.........|.........|.........|.........|.........|.........|.........|.........|.........| 7000
.........|.........|.........|.........|.........|.........|.........|.........|.........|.........| 8000
.........|.........|.........|.........|.........|.........|.........|.........|.........|.........| 9000
.........|.........|.........|.........|......

In [53]:
len(data)

10000

In [54]:
data[0]

(datetime.datetime(2019, 2, 1, 9, 32, 19, 55006),
 datetime.datetime(2019, 2, 1, 9, 32, 19, 497000),
 datetime.datetime(2019, 2, 1, 9, 32, 19, 501059),
 62401)

In [55]:
counts = [d[3] for d in data]

In [57]:
sorted(counts)[:10]

[62401, 62402, 62403, 62404, 62405, 62406, 62407, 62408, 62409, 62410]

In [58]:
sorted(counts)[-10:]

[72391, 72392, 72393, 72394, 72395, 72396, 72397, 72398, 72399, 72400]

In [59]:
data[-1]

(datetime.datetime(2019, 2, 1, 9, 33, 3, 205188),
 datetime.datetime(2019, 2, 1, 9, 33, 3, 723000),
 datetime.datetime(2019, 2, 1, 9, 33, 3, 731460),
 72397)

datetime.datetime(2019, 1, 31, 17, 25, 3, 622541)

In [15]:
import datetime

In [19]:
datetime.datetime.fromtimestamp(kafka_time/1000)

datetime.datetime(2019, 1, 31, 17, 25, 3, 943000)